# Exemples de fonctions Pandas utiles pour le TP 1

## Importations des librairies utiles

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import mutual_info_regression
from sklearn.metrics import mean_squared_error, r2_score
print('numpy %s, pandas %s'%(np.__version__,pd.__version__))
print(os.getcwd())


## Creation depuis un CSV et affichage d'un Dataframe Pandas

In [ ]:
my_dataframe = pd.read_csv('data/iris.csv')
my_dataframe
# display(my_dataframe)
# print("fin")


## Transformation de l'ID en index

In [ ]:
my_dataframe.set_index('id_Iris',inplace=True)
my_dataframe


## Tri selon l'index

In [ ]:
my_dataframe.sort_index(inplace=True) #Inplace permet de conserver les modification du DataFrame
my_dataframe


## Exploration des données / Importance des Features

In [ ]:
my_dataframe.info()


In [ ]:
my_dataframe.describe()


In [ ]:
# Calcul du quantile 0.9 (90% des valeurs sont dessous)
my_dataframe.quantile(0.9)


### Création de catégories pour les tailles et ajout du nom de l'iris

In [ ]:
# Créer une catégorie à partir des dimensions en cm
for taille in ["Longueur Sépale (cm)","Largeur Sépale (cm)","Longueur Pétale (cm)","Largeur Pétale (cm)"]:
    my_dataframe[taille.replace(" (cm)","")] = pd.qcut(my_dataframe[taille],3,['petit','moyen','grand'])

# Ajouter le dnom d'Iris à partir des dimensions en cm
my_dataframe['Nom Iris'] = my_dataframe['Type Iris'].map({
    0:'Setosa',
    1:'Versicolor',
    2:'Virginica',
})

# Affichage
display(my_dataframe)
my_dataframe.info()


### Distribution des labels selon une feature

In [ ]:
display(pd.crosstab(my_dataframe["Longueur Pétale"],my_dataframe["Nom Iris"]))
print("-"*50)
display(pd.crosstab(my_dataframe["Largeur Sépale"],my_dataframe["Nom Iris"]))


### Information Mutuelle

Identification des features discrets

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_regression.html


studied_features = ["Longueur Sépale (cm)","Largeur Sépale (cm)","Longueur Pétale","Largeur Pétale"]
X = my_dataframe[studied_features].copy()
for colname in X.columns:
    if X[colname].dtypes=='category':
        X[colname], _ = X[colname].factorize() # transforme des catégories en nombre entier
    else:
         X[colname].fillna(X[colname].mean(), inplace=True) # Si numérique, Remplacement des valeurs vides par la moyenne.

discrete_features = X.dtypes == "int64" # "Génère la liste indiquant features discrets ou continus"
print("discrete_features:\n",discrete_features)


Calcul des scores d'information mutuelle

In [ ]:
mi_scores = mutual_info_regression(
    X,
    my_dataframe['Type Iris'],
    discrete_features = discrete_features
)
mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)
mi_scores


Visualisation de l'information mutuelle

In [ ]:
scores = mi_scores.sort_values(ascending=True)
width = np.arange(len(scores))
ticks = list(scores.index)
plt.barh(width, scores)
plt.yticks(width, ticks)
plt.title("Mutual Information Scores")


## Filtrage des données

In [ ]:
my_dataframe.boxplot(fontsize=12, figsize=(18,10))


Exclusion des outliers

In [ ]:
# filtered_dataframe
exclusion_outliers = my_dataframe["Largeur Sépale (cm)"].between(2.1,4)
print("Total:",len(exclusion_outliers), "Conservés:", exclusion_outliers.sum())
filtered_dataframe =  my_dataframe[exclusion_outliers]
filtered_dataframe.boxplot(fontsize=12, figsize=(18,10))


### Création d'un petit Dataframe avec des données manquantes dans "Longueur Sépale"

In [ ]:
# Sélection des Iris avec Longueur Sépale indéfinie
small_df_na = my_dataframe[my_dataframe['Longueur Sépale (cm)'].isna()]

# Sélection aléatoire de 20 Iris avec Longueur Sépale définie
small_df_rnd = my_dataframe[~my_dataframe['Longueur Sépale (cm)'].isna()].sample(20, random_state=42)
my_dataframe[my_dataframe['Longueur Sépale (cm)'].isna()]
small_df = pd.concat([small_df_na,small_df_rnd]).sort_index()
small_df


### Suppression des lignes avec des données manquantes

In [ ]:
small_df[~small_df.isna().any(axis=1)]


### Suppression d'une colonne

In [ ]:
small_df.drop('Longueur Sépale (cm)', axis=1)


### Remplacement par la moyenne

In [ ]:
fulfilled_small_df=small_df.copy()
fulfilled_small_df["Longueur Sépale (cm)"].fillna(small_df["Longueur Sépale (cm)"].mean(),inplace=True)
fulfilled_small_df


## Gestion des données manquantes (Avancé)

### Interpolation linéaire

Régression linéaire entre Longueur Sépale et Longueur Pétale

In [ ]:
# Création d'un filtre pour sélectionner les valeurs non nulles
filter_not_nan = ~(small_df['Longueur Sépale (cm)'].isnull() | small_df['Longueur Pétale (cm)'].isnull())
print("---- filter_not_nan -----")
display(filter_not_nan)

# Création des deux variables X et y pour la régression
# reshape permet de créer "un tableau de tableau" pour le fit, car la méthode attend un nombre quelconque d'entrées dans x.
x = small_df.loc[filter_not_nan,'Longueur Pétale (cm)'].values.reshape(-1, 1)
print("---- x -----")
display(x)
y = small_df.loc[filter_not_nan,'Longueur Sépale (cm)']

# Création, appretissage et inférence du modèle de régression
model = LinearRegression()
model.fit(x,y)
y_predict = model.predict(x)

# Visualisation
plt.scatter(x, y)
plt.plot(x, y_predict, color='g')
plt.xlabel("Longueur Pétale (cm)")
plt.ylabel("Longueur Sépale (cm)")
plt.title("y=%.2fx + %.2f" % (model.coef_, model.intercept_))
plt.show()


Création d'une interpolation pour les longueurs sépales manquante.

In [ ]:
x_reg = small_df.loc[small_df['Longueur Sépale (cm)'].isnull() ,'Longueur Pétale (cm)'].values.reshape(-1, 1)
y_reg = model.predict(x_reg)
y_reg


### Affichage des points calculé par régression linéaire

In [ ]:
plt.scatter(x, y)
plt.scatter(x_reg, y_reg, color='r')
plt.plot(x, y_predict, color='g')
plt.xlabel("Longueur Pétale (cm)")
plt.ylabel("Longueur Sépale (cm)")
plt.title("y=%.2fx + %.2f" % (model.coef_, model.intercept_))
plt.show()
fulfilled_small_df.loc[small_df['Longueur Sépale (cm)'].isnull() ,'Longueur Sépale (cm)'] = y_reg
fulfilled_small_df


## Créer des Features

### compter des conditions (nombre de grande dimension)

In [ ]:
discrete_features = ["Longueur Sépale","Largeur Sépale","Longueur Pétale","Largeur Pétale"]
my_dataframe["nbGrand"] = (my_dataframe[discrete_features]=="grand").sum(axis=1)
display(my_dataframe.sample(5, random_state=42))
display(pd.crosstab(my_dataframe["nbGrand"],my_dataframe["Nom Iris"]))


### Group & Transform : Ajouter la largeur moyenne de la pétale en fonction de sa catégorie de longueur

In [ ]:
my_dataframe["Moy Largeur Pétale"] = (
    my_dataframe.groupby("Longueur Pétale") # Pour chaque catégorie de longueur de pétale.
    ["Largeur Pétale (cm)"]                 # Nous lisons la largeur de pétale...
    .transform("mean")                      # ...et nous calculons et ajoutons la moyenne
)
my_dataframe


## Target Encoding

#### Sans lissage

In [ ]:
my_dataframe["Moy Type Iris"] = (
    my_dataframe.groupby("Longueur Pétale")  # Pour chaque catégorie de longueur de pétale.
    ["Type Iris"]                            # Nous lisons le label
    .transform("mean")                       #... et nous calculons et ajoutons la moyenne
)
my_dataframe


### Avec Lissage

In [ ]:
# Facteur de lissage = 100
coef = 150/(150 + 100)
my_dataframe["Moy Type Iris"] = coef * my_dataframe["Moy Type Iris"] + (1-coef) * my_dataframe["Type Iris"].mean()
my_dataframe["Moy Type Iris"]
my_dataframe
